# Task 1 - Teoría

1. Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

2. Defina en sus propias palabras el término “Arc Consistency”

# Task 2 - CSP con Backtracking, Beam y Local Search

In [61]:
import time

## Definición de variables y restricciones

In [62]:
from itertools import combinations

dias = ['lunes', 'martes', 'miercoles']

# Definimos los exámenes y los estudiantes que toman cada examen
examenes = ['IA', 'ADA', 'COMPIS', 'CALCULO', 'FISICA', 'QUIMICA', 'BIOLOGIA']
estudiantes_por_examen = {
    'IA': ['Angel', 'Alejandro'],
    'ADA': ['Diego'],
    'COMPIS': ['Alejandro', 'Mark'],
    'CALCULO': ['Angel', 'Mark'],
    'FISICA': ['Mark', 'Diego'],
    'QUIMICA': ['Diego', 'Angel'],
    'BIOLOGIA': ['Alejandro'],
}


# Definimos las variables que representan los exámenes y sus posibles días
variables = {examen: dias for examen in examenes}


## Verificar si la asignación es válida

In [63]:
# Definimos la función para verificar si una asignación es válida
def esValida(asignacion):
    # Verificar que los estudiantes que toman el mismo curso no tengan examen el mismo día
    for examen1, examen2 in combinations(variables, 2):
        if asignacion[examen1] == asignacion[examen2]:
            estudiantes_examen1 = set(estudiantes_por_examen[examen1])
            estudiantes_examen2 = set(estudiantes_por_examen[examen2])
            if len(estudiantes_examen1.intersection(estudiantes_examen2)) > 0:
                return False

    # Verificar que ningún estudiante tenga más de un examen por día
    for dia in dias:
        examenesPorDia = [examen for examen, diaAsignado in asignacion.items() if diaAsignado == dia]
        estudiantesPorDia = set()
        for examen in examenesPorDia:
            estudiantesPorDia.update(estudiantes_por_examen[examen])
        if len(estudiantesPorDia) < len(examenesPorDia):
            return False

    return True

## BackTraking

In [64]:
class backTraking:
  def __init__(self, variables, dias, estudiantes_por_examen, esValida):
        self.variables = variables
        self.dias = dias
        self.estudiantes_por_examen = estudiantes_por_examen
        self.esValida = esValida
        self.asignacion = {}

  def backtracking_search(self):
    # Si la asignación es completa y válida, devolverla
    if set(self.asignacion.keys()) == set(self.variables) and self.esValida(self.asignacion):
        return self.asignacion

    # Seleccionar un examen sin asignar
    for examen in self.variables:
        if examen not in self.asignacion:
            # Para cada día, asignar ese día al examen y hacer una llamada recursiva
            for dia in self.dias:
                self.asignacion[examen] = dia
                result = self.backtracking_search()

                # Si la llamada recursiva devuelve una asignación válida, devolver esa asignación
                if result is not None:
                    return result

                # Si no, deshacer la asignación del día al examen
                del self.asignacion[examen]

    # Si ninguno de los días resulta en una asignación válida, devolver None
    return None

In [65]:
scheduler = backTraking(variables, dias, estudiantes_por_examen, esValida)
startTime = time.time()
result = scheduler.backtracking_search()
endTime = time.time()
print("Tiempo de ejecución: ", endTime - startTime)
print(result)

Tiempo de ejecución:  0.07100486755371094
{'IA': 'lunes', 'ADA': 'martes', 'COMPIS': 'miercoles', 'CALCULO': 'martes', 'FISICA': 'lunes', 'QUIMICA': 'miercoles', 'BIOLOGIA': 'martes'}


## Beam Search

In [66]:
# Redefinición de esValida. 
def es_valido(estado):
    for dia in dias:
        examenes_dia = [e for e, d in estado.items() if d == dia]
        estudiantes_por_dia = {}
        for examen in examenes_dia:
            estudiantes = estudiantes_por_examen[examen]
            for estudiante in estudiantes:
                if estudiante in estudiantes_por_dia:
                    return False  # Mismo estudiante con más de un examen en un día
                estudiantes_por_dia[estudiante] = True
    return True

In [67]:
# Función para evaluar un estado utilizando una heurística (en este caso, cantidad de restricciones incumplidas)
def evaluar_estado(estado):
    restricciones_incumplidas = 0
    estudiantes_por_dia_por_examen = {examen: {dia: [] for dia in dias} for examen in examenes}
    for examen, dia in estado.items():
        estudiantes = estudiantes_por_examen[examen]
        for estudiante in estudiantes:
            if estudiante in estudiantes_por_dia_por_examen[examen][dia]:
                restricciones_incumplidas += 1  # Mismo estudiante con más de un examen en un día
            estudiantes_por_dia_por_examen[examen][dia].append(estudiante)
    return -restricciones_incumplidas  # Queremos minimizar las restricciones incumplidas

In [68]:
from itertools import permutations

# Algoritmo de beam search
def beam_search(beam_width):
    mejor_estado = None
    mejor_puntaje = float('-inf')
    for permutacion in permutations(examenes):
        estado = {examen: None for examen in examenes}
        for i, examen in enumerate(permutacion):
            estado[examen] = dias[i % len(dias)]
        if es_valido(estado):
            puntaje_actual = evaluar_estado(estado)
            if puntaje_actual > mejor_puntaje:
                mejor_estado = estado
                mejor_puntaje = puntaje_actual
    return mejor_estado


In [69]:
startTime = time.time()
solucion = beam_search(5)
endTime = time.time()
print("Tiempo de ejecución: ", endTime - startTime)
print("Solución encontrada:")
print(solucion)

Tiempo de ejecución:  0.00899958610534668
Solución encontrada:
{'IA': 'martes', 'ADA': 'lunes', 'COMPIS': 'miercoles', 'CALCULO': 'lunes', 'FISICA': 'martes', 'QUIMICA': 'miercoles', 'BIOLOGIA': 'lunes'}


## Local Search

In [70]:
import random

class LocalSearch:
    def __init__(self, variables, dias, estudiantes_por_examen, esValida):
        self.variables = variables
        self.dias = dias
        self.estudiantes_por_examen = estudiantes_por_examen
        self.esValida = esValida
        self.asignacion = self.generar_asignacion_inicial()

    def generar_asignacion_inicial(self):
        # Generar una asignación inicial aleatoria
        asignacion = {}
        for examen in self.variables:
            asignacion[examen] = random.choice(self.dias)
        return asignacion

    def generar_vecinos(self):
        # Generar todas las soluciones vecinas cambiando un examen de día
        vecinos = []
        for examen in self.variables:
            for dia in self.dias:
                if self.asignacion[examen] != dia:
                    vecino = self.asignacion.copy()
                    vecino[examen] = dia
                    vecinos.append(vecino)
        return vecinos

    def local_search(self):
        while True:
            vecinos = self.generar_vecinos()
            # Ordenar los vecinos por su valor de evaluación
            vecinos.sort(key=self.esValida, reverse=True)
            # Si el mejor vecino es peor que la solución actual, devolver la solución actual
            if not self.esValida(vecinos[0]):
                return self.asignacion
            # Si no, actualizar la solución actual al mejor vecino
            self.asignacion = vecinos[0]

In [71]:
scheduler = LocalSearch(variables, dias, estudiantes_por_examen, esValida)
startTime = time.time()
result = scheduler.local_search()
endTime = time.time()
print("Tiempo de ejecución: ", endTime - startTime)
print(result)

Tiempo de ejecución:  0.0
{'IA': 'martes', 'ADA': 'miercoles', 'COMPIS': 'martes', 'CALCULO': 'martes', 'FISICA': 'miercoles', 'QUIMICA': 'martes', 'BIOLOGIA': 'martes'}


# Conclusiones